In [111]:
#Загрузка данных.
import pandas as pd
import numpy as np
df1 = pd.read_csv('b1.csv')
df2 = pd.read_csv('b2.csv')
df3 = pd.read_csv('b3.csv')
df4 = pd.read_csv('b4.csv')
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)
df3 = df3.drop('Unnamed: 0', axis=1)
df4 = df4.drop('Unnamed: 0', axis=1)

In [112]:
df1

,rate,id,A,A_errors,A_time,B,B_errors,B_time,C,C_errors,...,H_errors,H_time,I,I_errors,I_time,J,J_errors,J_time,ready_num,penalty
0,1,Денис Сухарев,1.0,0.0,0.000000,1.0,0.0,0.000142,1.0,0.0,...,0.0,0.000712,1.0,1.0,0.000855,1.0,0.0,0.000855,10,0.001904
1,2,Надежда,1.0,0.0,0.001994,1.0,0.0,0.002991,1.0,0.0,...,1.0,0.071652,1.0,1.0,0.080912,1.0,0.0,0.161681,10,0.048294
2,3,Dmitry,1.0,0.0,0.007835,1.0,0.0,0.000142,1.0,0.0,...,0.0,0.222222,1.0,1.0,0.290598,1.0,0.0,0.200427,10,0.088038
3,4,Рымарев Лев,1.0,0.0,0.000285,1.0,1.0,0.004986,1.0,0.0,...,0.0,0.042450,1.0,6.0,0.222222,NaN,NaN,NaN,10,0.106506
4,5,Алексей Подскрёбышев,1.0,1.0,0.014387,1.0,1.0,0.016809,1.0,2.0,...,3.0,0.178917,1.0,0.0,0.188319,1.0,0.0,0.410256,10,0.119616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,1944-2110,Kyle A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
2105,1944-2110,niki.ta.alex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
2106,1944-2110,Deffron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
2107,1944-2110,astro45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000


In [113]:
#Модель предсказывающая MAE, которая уччится на одном даатсете, а применяется на другом.
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

#columns_to_drop = ['A_errors', 'B_errors', 'C_errors', 'D_errors', 'E_errors', 'F_errors', 'G_errors', 'H_errors', 'I_errors', 'J_errors']
def MAE_model2(df1, df2, target):
    X_train = df1.drop(target, axis=1)
    X_train = X_train.drop('rate', axis=1)
    X_train = X_train.drop('id', axis=1)
    #X_train = X_train.drop(columns=columns_to_drop)
    y_train = df1[target]
    X_test = df2.drop(target, axis=1)
    X_test = X_test.drop('rate', axis=1)
    X_test = X_test.drop('id', axis=1)
    #X_test = X_test.drop(columns=columns_to_drop)
    y_test = df2[target]

    train_pool = Pool(
        data=X_train,
        label=y_train
    )

    test_pool = Pool(
        data=X_test,
        label=y_test
    )

    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.1,
        depth=3,
        loss_function='RMSE',
        random_seed=42,
        verbose=None
    )
    model.fit(train_pool, eval_set=test_pool, verbose=False)
    y_pred = model.predict(test_pool)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    return model

In [114]:
#Сохранили модель обученую на 1 и 2 наборах данных.
df12 = pd.concat([df1, df2], axis=0)
df123 = pd.concat([df12, df3], axis=0)
df1234 = pd.concat([df123, df4], axis=0)
num_model2 = MAE_model2(df1234, df4, 'ready_num')
num_model2.save_model("num_model2.cbm")

MSE: 0.3071
MAE: 0.3637


In [115]:
print('Если взять середину недели и попробовать предсказать, сколько задач человек решит к концу недели?')
print('Обучен на неделе i предсказывает неделю j кратко запишем, как i -> j')
print('1 -> 2')
MAE_model2(df1, df2, 'ready_num')
print('1 -> 3')
MAE_model2(df1, df3, 'ready_num')
print('1 -> 4')
MAE_model2(df1, df4, 'ready_num')
print('2 -> 3')
MAE_model2(df2, df3, 'ready_num')
print('2 -> 4')
MAE_model2(df2, df4, 'ready_num')
print('3 -> 4')
MAE_model2(df3, df4, 'ready_num')

print('12 -> 3')
MAE_model2(df12, df3, 'ready_num')
print('12 -> 4')
MAE_model2(df12, df4, 'ready_num')
print('123 -> 4')
MAE_model2(df123, df4, 'ready_num')

Если взять середину недели и попробовать предсказать, сколько задач человек решит к концу недели?
Обучен на неделе i предсказывает неделю j кратко запишем, как i -> j
1 -> 2
MSE: 0.6943
MAE: 0.5758
1 -> 3
MSE: 0.3619
MAE: 0.4283
1 -> 4
MSE: 0.6589
MAE: 0.5889
2 -> 3
MSE: 0.4731
MAE: 0.5190
2 -> 4
MSE: 0.4173
MAE: 0.4780
3 -> 4
MSE: 0.3996
MAE: 0.4618
12 -> 3
MSE: 0.3050
MAE: 0.4055
12 -> 4
MSE: 0.4471
MAE: 0.4823
123 -> 4
MSE: 0.3993
MAE: 0.4440
